In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import os

In [2]:
os.getcwd()

'/root/autodl-tmp'

In [3]:
# Folder Path
path = "/root/autodl-tmp/data/aclImdb/train/neg"
# Change the directory
os.chdir(path)

train_neg = []

# Read text File
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        train_neg.append(f.read())
  

# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)

In [4]:
path = "/root/autodl-tmp/data/aclImdb/train/pos"
# Change the directory
os.chdir(path)

train_pos = []

# Read text File
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        train_pos.append(f.read())
  

# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)

In [5]:
print(len(train_pos)) # number of positive comments
print(len(train_neg)) # number of negative comments

12500
12500


In [6]:
texts_train = train_neg + train_pos
y_train_neg = [0]*len(train_neg)
y_train_pos = [1]*len(train_pos)
y_train = np.array(y_train_neg + y_train_pos)

In [7]:
texts_train[12499] # negative

"I think my summary sums it up. I found it inane and stupid. I also saw the ending a mile a way. Everyone is copying that ending anymore when doing a TV/Theater crossover anymore. Sometimes, it's better to let the movie stand alone.<br /><br />Others, its better to forget the movie altogether. This is one of the others....\n"

In [8]:
texts_train[12500] # positive

'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.'

In [9]:
vocabulary = 10000
tokenizer = Tokenizer(num_words=vocabulary)
tokenizer.fit_on_texts(texts_train) # build dictionary

word_index = tokenizer.word_index # word dictionary
sequence_train = tokenizer.texts_to_sequences(texts_train) # text to sequence

In [10]:
# align sequences
word_num = 20
X_train = preprocessing.sequence.pad_sequences(sequence_train, maxlen=word_num) # Pre-padding or removing values from the beginning of the sequence is the default.

Preprocessing done (tokenization -> encoding -> alignment)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [12]:
embedding_dim = 8
model = Sequential()
model.add(Embedding(vocabulary, embedding_dim, input_length=word_num))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


2022-03-05 15:26:22.833277: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 15:26:23.545506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22312 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3e:00.0, compute capability: 8.6


In [13]:
epochs = 50
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['acc'])

/root/miniconda3/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
history = model.fit(X_train,y_train, epochs=epochs, batch_size=32, validation_data=(X_valid,y_valid))

Epoch 1/50
 93/625 [===>..........................] - ETA: 0s - loss: 0.6930 - acc: 0.5114 

2022-03-05 15:26:25.395588: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


625/625 [==============================] - 3s 2ms/step - loss: 0.6928 - acc: 0.5103 - val_loss: 0.6922 - val_acc: 0.5212
Epoch 2/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6909 - acc: 0.5459 - val_loss: 0.6912 - val_acc: 0.5392
Epoch 3/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6885 - acc: 0.5812 - val_loss: 0.6896 - val_acc: 0.5606
Epoch 4/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6853 - acc: 0.6166 - val_loss: 0.6872 - val_acc: 0.5840
Epoch 5/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6810 - acc: 0.6535 - val_loss: 0.6835 - val_acc: 0.6062
Epoch 6/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6749 - acc: 0.6838 - val_loss: 0.6782 - val_acc: 0.6346
Epoch 7/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6669 - acc: 0.7046 - val_loss: 0.6708 - val_acc: 0.6582
Epoch 8/50
625/625 [==============================] - 1s 2ms/step - loss: 0.6566 - acc:

In [15]:
comments = [texts_train[610]]
comments # should be negative

["I found this film to be quite an oddity. From the very get go I found it extremely hard to like this movie, and now after a little thinking about it I can pretty much pinpoint the reason why. Jean-Marc Barr, although I love him to bits (I think Zentropa is one of the best movies ever made) is quite miscast here, and although I can't figure for the life of me who would be better, I am sure someone could have taken his place quite easily and make this film work. Everything else is fine, except for the stabs at weak comedy (A Meet The Parents Joke is not really needed, filmmakers!) and I really like Richard E. Grant as the British Major. It just suffers from one thing.. Jean-Marc."]

In [16]:
sequence_prediction = tokenizer.texts_to_sequences(comments) # input should be list of sequences
xhat = preprocessing.sequence.pad_sequences(sequence_prediction, maxlen=word_num)
model.predict(xhat)

array([[0.35534427]], dtype=float32)